In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!nvidia-smi

Mon Dec 27 02:37:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2531277352792400979
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11320098816
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16835937091164972347
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
 xla_global_id: 416903419]

In [5]:
import pandas as pd
import numpy as np

In [6]:
train = pd.read_csv('/content/drive/MyDrive/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/test.csv')
submission = pd.read_csv("/content/drive/MyDrive/data/sample_submission.csv")
train['data'][0]

'신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지 마시고\\n보편적으로 모든국민이 수긍할  수 있는 복지정책 펴 주시길 바랍니다.\\n저도 신혼부부이지만 당첨되는 사람 로또되는 이런주택정책 반대합니다.\\n국민세금을 일부 사람들에게 퍼주기식이 되면 안되죠..\\n그 세금으로 우리아이 안전하게 맡길 수 있는 보육시설을 전국에 설치해 주세요..\\n대기업들은 솔선수범해서 모든 사업장에 의무설치 할 수 있도록 하시구요..\\n집 보다 애 맡길데가 없어 경력단절 되는게 더 괴롭습니다.!\\n집은 개인의 능력을 키워 사는게 맞습니다.\\n그 능력을 키울수 있도록 육아 전담에 힘을 기울이는게 맞습니다.\\n우리아이 부모가 키우는거 맞지만 이제는 국가가\\n책임지는 시대로 가는게 맞다고 봅니다.\\n그렇잖아도 부동산 가격 자꾸 올라가는게 정부정책이 잘못 되었다고 봅니다.\\n부동산은 그냥 내버려 두세요!  좀!\\n건들수록 역효과네요..'

# 1. 전처리

In [7]:
train = train[train['data'].notnull()]
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39992 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     39992 non-null  int64 
 1   category  39992 non-null  int64 
 2   data      39992 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.2+ MB


In [8]:
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords = ['의','가','이','은','들','는','좀','잘','걍',
             '과','도','를','으로','자','에','와','한','하다','을',
             '더','좀','자꾸','그냥','이제','우리']

In [9]:
train['data'][0]

'신혼부부위한 주택정책 보다 보육시설 늘려주세요 국민세금으로 일부를 위한 정책펴지 마시고보편적으로 모든국민이 수긍할  수 있는 복지정책 펴 주시길 바랍니다저도 신혼부부이지만 당첨되는 사람 로또되는 이런주택정책 반대합니다국민세금을 일부 사람들에게 퍼주기식이 되면 안되죠그 세금으로 우리아이 안전하게 맡길 수 있는 보육시설을 전국에 설치해 주세요대기업들은 솔선수범해서 모든 사업장에 의무설치 할 수 있도록 하시구요집 보다 애 맡길데가 없어 경력단절 되는게 더 괴롭습니다집은 개인의 능력을 키워 사는게 맞습니다그 능력을 키울수 있도록 육아 전담에 힘을 기울이는게 맞습니다우리아이 부모가 키우는거 맞지만 이제는 국가가책임지는 시대로 가는게 맞다고 봅니다그렇잖아도 부동산 가격 자꾸 올라가는게 정부정책이 잘못 되었다고 봅니다부동산은 그냥 내버려 두세요  좀건들수록 역효과네요'

---

# 2. 데이터 변환
- train data -> train/validation data로 분할
- tsv 파일로 변환



- train --> train_train.tsv, train_validation.tsv
- test --> test.tsv

In [10]:
train_train = train[:30000]
train_validation = train[30000:]
train_validation

,index,category,data
30006,30006,0,고 장자영 사건관련 수첩 보도내용 일 오후 방송된 시사교양 프로그램 수첩은 고 ...
30007,30007,1,나경원을 파면 해주십시요 나경원을 파면 해주십시요
30008,30008,0,대부분의 초중 교사 초 중 교사가 하는 말 무슨 이유가 있든 여자는 건들면 안된다...
30009,30009,0,모든 국민의 양심적 병역거부와 병역선택의 보편화를 건의합니다 존경하는 대통령님 언제...
30010,30010,1,조두순 형량을 늘려주세요 조두순은 한 아이의 평생을 짓밟고 없애버린 희대의 범인입니...
...,...,...,...
39995,39995,2,시간유연근무제 저는 국립대에서 일하고 있는 비정규직 근로자입니다동시에 두 자녀의 ...
39996,39996,0,소년법을 폐지해 주시고 부산 여중생 가해자 학생들의 강력한 처벌을 요구합니다 소년법...
39997,39997,2,무서운데 지켜야 할게 있어요 도와주세요 안녕하세요 한부모엄마 입니다양육비를 받고 ...
39998,39998,2,교복에 고정식 이름표를 달게 하는 것을 금지해 주세요 교복에 이름표를 박아놓아 탈...


In [11]:
import os
os.chdir('/content/drive/MyDrive/data')
os.getcwd()

'/content/drive/MyDrive/data'

In [16]:
train_train.to_csv('train_train.tsv',sep='\t',encoding='utf-8',index=False)
train_validation.to_csv('train_validation.tsv',sep='\t',encoding='utf-8',index=False)

In [17]:
test['category'] = 0
test.to_csv('test.tsv',sep='\t',encoding='utf-8',index=False)
test

,index,data,category
0,0,소년법 폐지해주세요 법 아래에서 보호받아야 할 아이들이법으로 인해 보호받지 못하고 ...,0
1,1,국공립 유치원 증설에 관하여 국공립 유치원 부지 학보와건립및 증설에지역 어린이 놀이...,0
2,2,나경원파면 나경원의원의 동계올림픽 위원을 파면해 주세요,0
3,3,국민위원에가 삼성편만들어요 삼성에서 년간 일하고 혈암과 백혈병 진단을 받은 사람이...,0
4,4,방과후유치원어린이집 영어교육을 유지시켜주세요 저는 아이 셋 키우는 평범한 주부입니다...,0
...,...,...,...
4995,4995,페이스북 개인신상정보 개털렸는데 이대로 존경하신 문재인 대통령님 얼마전 미국본사로둔...,0
4996,4996,아시안 게임에 야구프로스포츠 선수참여금지 청원합니다 뭐 대놓고 병역브로커 하자는것인...,0
4997,4997,손흥민국가대표자격 영구박탈청원합니다 광고찍을때랑 군면제걸렸을때 소속팀 토튼햄에서만 ...,0
4998,4998,한진 사모님 이명희를 구속시키는 방안을 찿아보세요 일단 밝혀진 피해자가 명입니다 아...,0


---
# 3. Kobert 모델



### 1) install & setting

In [19]:
os.getcwd()

'/content/drive/My Drive/data'

In [20]:
os.chdir('/content/drive/MyDrive/data/KoBERT-master')
os.getcwd()

'/content/drive/MyDrive/data/KoBERT-master'

In [26]:
pip install -r requirements.txt

In [27]:
pip install .

Processing /content/drive/MyDrive/data/KoBERT-master
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for kobert: filename=kobert-0.2.1-py3-none-any.whl size=13006 sha256=6ff62c4e21762093f4b25c1c4d72e5d7ca68bd9c78fe65e2825e045936dd691b
  Stored in directory: /root/.cache/pip/wheels/21/20/5e/6fb72e5b281231254873585d6b140dee9b0d2c64c136df630a
Successfully built kobert
  Attempting uninstall: kobert
    Found existing installation: kobert 0.2.1
    Uninstalling kobert-0.2.1:
      Successfully uninstalled kobert-0.2.1


In [21]:
# pip install --upgrade sagemaker

In [22]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer # error 발생할 경우 런타임 다시 시작
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [35]:
device = torch.device("cuda:0")

In [36]:
#bert 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/drive/My Drive/data/KoBERT-master/.cache/kobert_v1.zip
using cached model. /content/drive/My Drive/data/KoBERT-master/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


### 2) 데이터 변환

In [37]:
train[:5]

,index,category,data
0,0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요 국민세금으로 일부를 위한 정책펴지 마...
1,1,0,학교이름에 남자도 붙여주세요 울산여자중학교에 재학중인 학생입니다 최근 양성평등 글짓...
2,2,1,빙상연맹 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요 최근 동계올림픽에...
3,3,1,티비 세세 관람가도 연령확인 의무화 하자 제기 에전에 티비를 보다가 잠시 딴일이 생...
4,4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야 무더운 여름철에는 남성들도 노넥타이...


In [38]:
dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/data/train_train.tsv', 
                                    field_indices=[2,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/drive/MyDrive/data/train_validation.tsv', 
                                   field_indices=[2,1], num_discard_samples=1)


In [39]:
dataset_train[0]

['신혼부부위한 주택정책 보다 보육시설 늘려주세요 국민세금으로 일부를 위한 정책펴지 마시고보편적으로 모든국민이 수긍할  수 있는 복지정책 펴 주시길 바랍니다저도 신혼부부이지만 당첨되는 사람 로또되는 이런주택정책 반대합니다국민세금을 일부 사람들에게 퍼주기식이 되면 안되죠그 세금으로 우리아이 안전하게 맡길 수 있는 보육시설을 전국에 설치해 주세요대기업들은 솔선수범해서 모든 사업장에 의무설치 할 수 있도록 하시구요집 보다 애 맡길데가 없어 경력단절 되는게 더 괴롭습니다집은 개인의 능력을 키워 사는게 맞습니다그 능력을 키울수 있도록 육아 전담에 힘을 기울이는게 맞습니다우리아이 부모가 키우는거 맞지만 이제는 국가가책임지는 시대로 가는게 맞다고 봅니다그렇잖아도 부동산 가격 자꾸 올라가는게 정부정책이 잘못 되었다고 봅니다부동산은 그냥 내버려 두세요  좀건들수록 역효과네요',
 '2']

In [40]:
## Setting parameters
max_len = 500
batch_size = 2
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [44]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [45]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/drive/My Drive/data/KoBERT-master/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [46]:
#배치 및 데이터로더 설정
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


---

### 3) 모델 학습

In [47]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3, ##주의: 클래스 수 바꾸어 주세요!##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [48]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device) #gpu

In [49]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [50]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 학습

In [ ]:
for e in range(num_epochs):
    torch.cuda.empty_cache()
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        # copiedOut = out.data.cpu().numpy()
        # del out
        # torch.cuda.empty_cache()

        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/15000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.4836046695709229 train acc 0.5
epoch 1 batch id 201 loss 0.9606243371963501 train acc 0.32338308457711445
epoch 1 batch id 401 loss 1.090392827987671 train acc 0.3341645885286783
epoch 1 batch id 601 loss 0.9149004220962524 train acc 0.3752079866888519
epoch 1 batch id 801 loss 0.6198902130126953 train acc 0.4307116104868914
epoch 1 batch id 1001 loss 0.23215359449386597 train acc 0.4945054945054945
epoch 1 batch id 1201 loss 0.25908270478248596 train acc 0.5424646128226478
epoch 1 batch id 1401 loss 0.029026519507169724 train acc 0.5852962169878658
epoch 1 batch id 1601 loss 0.024683753028512 train acc 0.6174266083697689
epoch 1 batch id 1801 loss 0.00945383682847023 train acc 0.6399222654081066
epoch 1 batch id 2001 loss 0.009697494097054005 train acc 0.6606696651674163
epoch 1 batch id 2201 loss 0.01929604820907116 train acc 0.6774193548387096
epoch 1 batch id 2401 loss 0.01265941746532917 train acc 0.6899208663057059
epoch 1 batch id 2601 loss 0.0040050642

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 10.5 GB  | Proc size: 5.3 GB
GPU RAM Free: 4184MB | Used: 7257MB | Util  63% | Total 11441MB


---
# 테스트
- category 임의로 생성

In [ ]:
new_test = nlp.data.TSVDataset('/content/drive/MyDrive/data/test.tsv', field_indices=[1,2], num_discard_samples=1)
test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

In [ ]:
pred = []
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
  token_ids = token_ids.long().to(device) 
  segment_ids = segment_ids.long().to(device) 
  valid_length= valid_length 
  out = model(token_ids, valid_length, segment_ids)
  prediction = out.cpu().detach().numpy().argmax()
  print(batch_id + "번째 문장의 분류 예측값은" + prediction + "입니다.")
  pred.append(prediction)